In [ ]:
import matplotlib.pyplot as plt

# ... (your existing code)
# Epoch 100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from sklearn.metrics import classification_report
import os
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/content/drive/MyDrive/Kaggle Train'
validation_data_dir = '/content/drive/MyDrive/Kaggle Validation'
nb_train_samples = 74
nb_validation_samples = 38
epochs = 100
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

def test_model(model, test_data_dir, batch_size):
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

    # Evaluate the model on the test data
    test_loss, test_accuracy = model.evaluate_generator(test_generator, steps=len(test_generator))
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy * 100}%")

    # Predict classes for the test data
    y_true = test_generator.classes
    y_pred = model.predict_generator(test_generator, steps=len(test_generator))
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]

    # Calculate and print precision, recall, and F1-score
    report = classification_report(y_true, y_pred, target_names=['class 0', 'class 1'])
    print("Classification Report:\n", report)

# Define the directory containing your test data
test_data_dir = '/content/drive/MyDrive/Kaggle Test'

# Call the test_model function to evaluate your model on the test data and calculate metrics
test_model(model, test_data_dir, batch_size)

# After training, you can use the model to predict whether an image is real or deepfake
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    #print(img)
    prediction = model.predict(img)
    print(prediction)
    return prediction[0][0]

# Example usage:
image_path_to_predict = '/content/drive/MyDrive/Real Images Dir/Images/LJ049-0079.png'
prediction = predict_image(image_path_to_predict)
if prediction >= 0.5000:
    print("Real Image")
    print("There for it's an audio image of real audio.")
else:
    print("Deepfake Image")
    print("There for it's an audio image of deep fake audio.")
# Create lists to store training and validation accuracy
train_acc_history = []
val_acc_history = []

# ... (your existing code)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

# Append accuracy values to the lists
train_acc_history.extend(history.history['accuracy'])
val_acc_history.extend(history.history['val_accuracy'])

# Plotting the accuracy graph
plt.plot(range(1, epochs + 1), train_acc_history, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_acc_history, label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()